In [3]:
import os
import datetime
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# Determine the start and end dates for the previous month
def last_month():
    end_date = datetime.datetime.now().replace(day=1) - datetime.timedelta(days=1) 
    start_date = end_date.replace(day=1) - datetime.timedelta(days=1)
    end_date += datetime.timedelta(days=1)
    return start_date, end_date

prev_month_start, prev_month_end = last_month()

# Determine the start and end dates for the current month
def current_month():
    # Get the current date
    today = datetime.datetime.now()

    # Calculate the first day of the current month
    first_day = today.replace(day=1)

    # Calculate the last day of the current month
    next_month = (today.replace(day=28) + datetime.timedelta(days=4))  # Mov|e to the 28th to ensure we're in the next month
    last_day = next_month - datetime.timedelta(days=next_month.day)

    return first_day, last_day

current_month_start, current_month_end = current_month()

In [4]:
def get_google_calendar_service():
    SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]
    
    # Load credentials from the file if available; otherwise, authenticate the user
    credentials = None
    token_path = "token.json"  # Change this to your token file path
    
    if os.path.exists(token_path):
        credentials = Credentials.from_authorized_user_file(token_path)
    
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            credentials = flow.run_local_server(port=0)
        
        with open(token_path, "w") as token:
            token.write(credentials.to_json())
    
    # Build the Google Calendar service
    service = build("calendar", "v3", credentials=credentials)
    return service

In [5]:
service = get_google_calendar_service()

In [6]:
from enum import IntEnum, EnumMeta

class PriceEnum(EnumMeta):
    """
    Prices per exercise
    """
    SOLO_PRICE = 180
    TWO_PRICE = 250
    THREE_PRICE = 300
    GROUP_PRICE = 70
    def __getitem__(cls, item):
        event_names, color_id = item['names'], item['colorId']
        
        participants = event_names.split(';')
        participants_num = len(participants)
                
        match participants_num:
            case 1:
                return cls.SOLO_PRICE
            case 2:
                return cls.TWO_PRICE
            case par if par >= 3: 
                return cls.THREE_PRICE
                
class Prices(IntEnum, metaclass=PriceEnum):
    pass
       

In [7]:
from enum import Enum
class ExerciseTypeColor(Enum):
    """
    Color Id per card group, there are trainees who buy 10 exercises and others who buy 15 exercises.
    """
    PERSONAL = '8'
    GROUP = '3'
    PILATES_10_CARD = '1'
    PILATES_15_CARD = '11'

    @classmethod
    def list(cls):
        return list(map(lambda e: e.value, cls))

In [8]:
def get_events(service, time_min, time_max) -> list:   
    """
    Use Google calendar API to retrieve ALL events between time_min and time_max.
    """
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=time_min.isoformat() + "Z",
            timeMax=time_max.isoformat() + "Z",
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])
    return events

In [68]:
def get_events_by_color_id(events, *color_ids) -> list:
    """
    Filter events of color_id in *color_ids.
    """
    events_by_color = []
    
    for event in events:
        if 'colorId' in event and event['colorId'] in color_ids:
            events_by_color.append(event)
    
    return events_by_color




def filter_events(events, condition) -> list:
    return filter(condition, events)

def filter_events_by_color_id(events, *color_ids):
    return filter_events(events, lambda event: 'colorId' in event and event['colorId'] in color_ids)

In [69]:
def extract_events_data(events) -> list:
    """
    
    """
    events_data = []
    
    for event in events:
        date = event["start"]["dateTime"].split('T')[0]
        time = event["start"]["dateTime"].split('T')[1].split('+')[0]
        color_id = event['colorId'] 
        
        if event['summary']:
            names_list = [name.strip() for name in event['summary'].split(';') if name.strip()]
            names = ', '.join(names_list)

            event_data = {
                'names': names,
                'date': date,
                'time': time,
                'colorId': color_id
            }

            events_data.append(event_data)

    return events_data


# def format_events(events) -> list:
    

In [12]:
def count_events(events) -> list:
    event_counts = []

    for event in events:
        event_summary_names, color_id = event.get("summary"), event.get("colorId")

        # Check if the event_summary_names is already in event_counts
        existing_event = next((e for e in event_counts if e["names"] == event_summary_names), None)

        if existing_event is None:
            # If not, add a new entry for the event_summary_names
            event_count_dict = {
                'names': event_summary_names,
                'count': 1,
                'colorId': color_id
            }
            
            event_counts.append(event_count_dict)
        else:
            # If it exists, increment the count for that event_summary_names
            existing_event["count"] += 1

    return event_counts

In [13]:
prev_month_events = get_events(service, time_min=current_month_start, time_max=current_month_end)

In [76]:
import itertools
from operator import itemgetter

def group_events_by_name(events) -> list:
    """
    Create a list of event and how many times they appear. Group by event name and colorId.
    """
    events_count = []
    summary_color_id_getter = itemgetter('summary')
    events = filter(lambda e: 'colorId' in e, events)
    for name, g in itertools.groupby(sorted(events, key=summary_color_id_getter), summary_color_id_getter):
        events_count.append({
                'names': name,
                'count': sum(1 for _ in g)
            })
    return events_count

In [108]:
def expand_exercises(event_counts) -> list:
    return [{'names': name, 'count': event.get('count')} for event in event_counts for name in event['names'].split(';')]


In [109]:
def calculate_group():
    pass

In [120]:
def calculate_personal():
    payment_period_events = get_events(service, time_min=current_month_start, time_max=current_month_end)
    personal_events = filter_events_by_color_id(payment_period_events, ExerciseTypeColor.PERSONAL.value)
    count_personal_events = group_events_by_name(personal_events)
    # why expand personal?
    solo_personal_events = expand_exercises(count_personal_events)
    
    
    
    

In [119]:
calculate_personal()

[{'names': 'אולגה', 'count': 4}, {'names': 'דורון איתמר וחמדה', 'count': 4}, {'names': 'כרמית', 'count': 3}, {'names': 'נטעלי', 'count': 3}, {'names': 'לירון', 'count': 3}, {'names': 'ליטל', 'count': 4}, {'names': 'נופר', 'count': 4}, {'names': 'דנית', 'count': 4}, {'names': 'נועה', 'count': 4}, {'names': 'רוית', 'count': 4}, {'names': 'דנית', 'count': 4}, {'names': 'נופר', 'count': 3}, {'names': 'ליטל', 'count': 3}, {'names': 'רותי', 'count': 3}, {'names': 'סימה', 'count': 1}, {'names': 'רותי משעלי', 'count': 1}, {'names': 'זהבה', 'count': 1}, {'names': 'סימה', 'count': 3}, {'names': 'רנא', 'count': 3}, {'names': '', 'count': 3}, {'names': 'שולה', 'count': 1}, {'names': '', 'count': 1}, {'names': 'לילי', 'count': 1}, {'names': 'לירון', 'count': 1}, {'names': 'שולה', 'count': 2}, {'names': 'עירית', 'count': 2}, {'names': 'לילי', 'count': 2}, {'names': 'שולה', 'count': 1}, {'names': 'צילה', 'count': 1}, {'names': 'לילי', 'count': 1}]


In [ ]:
def calculate(type: ExerciseTypeColor):
    match type:
        case ExerciseTypeColor.GROUP.value:
        case ExerciseTypeColor.PERSONAL.value:
        case ExerciseTypeColor.PILATES_10_CARD.value:
        case ExerciseTypeColor.PILATES_15_CARD.value:
            

In [22]:
def calculate_charges(event_counts) -> list:
    """
    Calclate how much each trainee should pay.
    """
    event_charges = []
    
    for event in event_counts:
        event_names, count, color_id = event.values()
        event_names_list = [name.strip() for name in event_names.split(';') if name.strip()]

        charge_for_event = Prices[event]
        
        for event_name in event_names_list:
            existing_event = next((e for e in event_charges if e['name'] == event_name), None)

            if existing_event is None:
                event_charges_dict = {
                    'name': event_name,
                    'amount': charge_for_event * count,
                    'pilates_card': 0
                }

                if color_id in [ExerciseTypeColor.PILATES_10_CARD.value, ExerciseTypeColor.PILATES_15_CARD.value]:
                    card_value = 10 if color_id == ExerciseTypeColor.PILATES_10_CARD.value else 15
                    event_charges_dict['pilates_card'] = card_value
                    event_charges_dict['amount'] += 750 if card_value == 10 else 1000

                event_charges.append(event_charges_dict)
            else:
                existing_event['amount'] += charge_for_event * count

                if color_id in [ExerciseTypeColor.PILATES_10_CARD.value, ExerciseTypeColor.PILATES_15_CARD.value] and not existing_event['pilates_card']:
                    card_value = 10 if color_id == ExerciseTypeColor.PILATES_10_CARD.value else 15
                    existing_event['pilates_card'] = card_value
                    existing_event['amount'] += 750 if card_value == 10 else 1000

    return event_charges

In [23]:
def calculate_pilates_card(event_counts):

    pilates_counts = []

    for event in event_counts:
        event_names, count, color_id = event.values()
        event_names_list = [name.strip() for name in event_names.split(';') if name.strip()]

        if color_id in [ExerciseTypeColor.PILATES_10_CARD.value, ExerciseTypeColor.PILATES_15_CARD.value]:
            for event_name in event_names_list:
                existing_event = next((e for e in pilates_counts if e['name'] == event_name), None)

                if existing_event is None:

                    match color_id:
                        case ExerciseTypeColor.PILATES_10_CARD.value:
                            card_value = 10
                        case ExerciseTypeColor.PILATES_15_CARD.value:
                            card_value = 15

                    pilates_counts_dict = {
                        'name': event_name,
                        'pilates_card': card_value,
                        'card_count': count,
                        'card_left': card_value - count
                    }
                    
                    pilates_counts.append(pilates_counts_dict)
                else: 
                    existing_event['card_count'] += count
                    existing_event['card_left'] = existing_event['pilates_card'] - existing_event['card_count']
                    
    return pilates_counts

In [24]:
import tkinter as tk
from tkinter import ttk

def get_row_color(color_id):
    # Map colorId to a corresponding pastel color for the row
    color_map = {
        '1': '#FFB6C1',  # lightpink
        '3': '#98FB98',  # palegreen
        '8': '#ADD8E6',  # lightblue
        '11': '#FFDAB9'  # peachpuff
    }
    return color_map.get(color_id, 'white')

def create_data_table(root, data, row, column, label_text, column_span=3):
    frame = ttk.Frame(root)
    frame.grid(row=row, column=column, sticky='nsew')

    label = tk.Label(frame, text=label_text, font=('Helvetica', 12, 'bold'))
    label.grid(row=0, column=0, pady=5, columnspan=column_span)

    tree = ttk.Treeview(frame, columns=('number', 'names', 'date', 'time'), show='headings')

    tree.heading('number', text='#')
    tree.heading('names', text='Names')
    tree.heading('date', text='Date')
    tree.heading('time', text='Time')

    tree.column('number', width=50)  
    tree.column('names', width=250)   
    tree.column('date', width=150)   
    tree.column('time', width=150)

    for col in ('number', 'names', 'date', 'time'):
        tree.column(col, anchor='center')

    row_number = 1
    for item in data:
        tree.insert('', 'end', values=(row_number, item['names'], item['date'], item['time']), tags=(item['colorId'],))
        row_number += 1

    for color_id in set(item['colorId'] for item in data):
        tree.tag_configure(color_id, background=get_row_color(color_id), foreground='black')

    tree.grid(row=1, column=0, padx=5, columnspan=column_span, sticky='nsew')
    

def create_count_table(root, data, row, column, label_text, column_span=3):
    frame = ttk.Frame(root)
    frame.grid(row=row, column=column, sticky='nsew')

    label = tk.Label(frame, text=label_text, font=('Helvetica', 12, 'bold'))
    label.grid(row=0, column=0, pady=5, columnspan=column_span)

    tree = ttk.Treeview(frame, columns=('number', 'name', 'amount'), show='headings')

    tree.heading('number', text='#')
    tree.heading('name', text='Name')
    tree.heading('amount', text='Amount')

    tree.column('number', width=50)  
    tree.column('name', width=150)
    tree.column('amount', width=50)  

    for col in ('number', 'name', 'amount'):
        tree.column(col, anchor='center')
        
    row_number = 1
    for item in data:
        tree.insert('', 'end', values=(row_number, item['name'], item['amount']))
        row_number += 1

    tree.grid(row=1, column=0, columnspan=column_span, sticky='nsew')
    
# Create the main Tkinter window
root = tk.Tk()
root.title("Data Tables")

# Get screen width and height
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# Set the window size 
window_width = int(screen_width * 1)
window_height = int(screen_height * 1)
root.geometry(f"{window_width}x{window_height}")

# Center the window on the screen
x_position = (screen_width - window_width) // 2
y_position = (screen_height - window_height) // 2
root.geometry(f"+{x_position}+{y_position}")

''

In [25]:
if __name__ == "__main__":
    prev_month_events = get_events(service, time_min=current_month_start, time_max=current_month_end)
    all_events = get_events_by_color_id(prev_month_events, ExerciseTypeColor.PERSONAL.value, ExerciseTypeColor.GROUP.value, ExerciseTypeColor.PILATES_10_CARD.value, ExerciseTypeColor.PILATES_15_CARD.value)
    personal_events = get_events_by_color_id(prev_month_events, ExerciseTypeColor.PERSONAL.value)
    group_events = get_events_by_color_id(prev_month_events, ExerciseTypeColor.GROUP.value)
    pilates_card_10_events = get_events_by_color_id(prev_month_events, ExerciseTypeColor.PILATES_10_CARD.value)
    pilates_card_15_events = get_events_by_color_id(prev_month_events, ExerciseTypeColor.PILATES_15_CARD.value)

    count = count_events(pilates_card_10_events)
    count_cal = calculate_charges(count)

    print(count_cal)

[{'name': 'שולה', 'amount': 1950, 'pilates_card': 10}, {'name': 'לילי', 'amount': 1950, 'pilates_card': 10}, {'name': 'לירון', 'amount': 1050, 'pilates_card': 10}, {'name': 'סימה', 'amount': 1950, 'pilates_card': 10}, {'name': 'רותי משעלי', 'amount': 1050, 'pilates_card': 10}, {'name': 'זהבה', 'amount': 1050, 'pilates_card': 10}, {'name': 'ליטל', 'amount': 2850, 'pilates_card': 10}, {'name': 'נופר', 'amount': 2850, 'pilates_card': 10}, {'name': 'דנית', 'amount': 3150, 'pilates_card': 10}, {'name': 'נועה', 'amount': 1950, 'pilates_card': 10}, {'name': 'רוית', 'amount': 1950, 'pilates_card': 10}, {'name': 'אולגה', 'amount': 1470, 'pilates_card': 10}, {'name': 'דורון איתמר וחמדה', 'amount': 1470, 'pilates_card': 10}, {'name': 'צילה', 'amount': 1050, 'pilates_card': 10}, {'name': 'רנא', 'amount': 1650, 'pilates_card': 10}, {'name': 'רותי', 'amount': 1650, 'pilates_card': 10}, {'name': 'עירית', 'amount': 1350, 'pilates_card': 10}]


In [26]:
    all_data = extract_events_data(all_events)
    personal_data = extract_events_data(personal_events)
    group_data = extract_events_data(group_events)
    pilates_card_10_data = extract_events_data(pilates_card_10_events)
    pilates_card_15_data = extract_events_data(pilates_card_15_events)

In [27]:
    # Gui
    create_data_table(root=root, data=personal_data, row=0, column=0, label_text="Personal Data")
    create_data_table(root=root, data=group_data, row=0, column=1, label_text="Group Data")
    create_data_table(root=root, data=pilates_card_10_data, row=1, column=0, label_text="Pilates 10 Card")
    create_data_table(root=root, data=pilates_card_15_data, row=1, column=1, label_text="Pilates 15 Card")
    create_data_table(root=root, data=all_data, row=2, column=0, label_text="All Data")

    create_count_table(root=root, data=count_cal, row=2, column=1, column_span=1, label_text="All Data")
    root.mainloop()